In [2]:
%reload_ext autoreload
%autoreload 2

import os
import time
from datetime import date, datetime, timedelta
from plotnine import *

os.chdir(f"/home/{os.getlogin()}/watttime-python-client-aer-algo")

import pandas as pd
from dateutil.parser import parse
from pytz import UTC, timezone
import matplotlib.pyplot as plt
from mizani.formatters import date_format

from watttime import (
    WattTimeMyAccess,
    WattTimeHistorical,
    WattTimeForecast,
    WattTimeOptimizer,
    RecalculatingWattTimeOptimizer, 
)

from evaluation import eval_framework as efu

The issue I am reproducing here is that when using the same start and end times, the `WatttimeOptimizer` and `RecalculatingWattTimeOptimizer` produce schedules of different time periods. Particularly, the `WatttimeOptimizer` schedule starts 5 hours later and covers a period 5 hours shorter than the `RecalculatingWattTimeOptimizer` schedule. My assumption is this has to do with a timezone adjustment.

In [5]:
region = "SPP_TX"
username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

In [8]:
start_time = datetime(2023, 7, 11, 20, 6)
start_time = UTC.localize(start_time)

In [ ]:
window_start = start_time
window_end = window_start + timedelta(hours = 12)

In [15]:
window_start

datetime.datetime(2023, 7, 11, 20, 6, tzinfo=<UTC>)

In [16]:
window_end

datetime.datetime(2023, 7, 12, 8, 6, tzinfo=<UTC>)

In [9]:
wt_opt = WattTimeOptimizer(username, password)
usage_power_kw = 12
moer_data = efu.get_historical_fcst_data(window_start, 12*12, region)

basic_usage_plan = wt_opt.get_optimal_usage_plan(
    region=region,
    usage_window_start=window_start,
    usage_window_end=window_end,
    usage_time_required_minutes=240,
    usage_power_kw=usage_power_kw,
    optimization_method="baseline",
    moer_data_override= moer_data
)

basic_usage_plan = basic_usage_plan.reset_index()

== Baseline fit! ==


In [10]:
basic_usage_plan

,point_time,pred_moer,usage,emissions_co2e_lb,energy_usage_mwh
0,2023-07-12 01:05:00+00:00,1347.8,5.0,1.3478,0.001
1,2023-07-12 01:10:00+00:00,1347.8,5.0,1.3478,0.001
2,2023-07-12 01:15:00+00:00,1342.8,5.0,1.3428,0.001
3,2023-07-12 01:20:00+00:00,1341.5,5.0,1.3415,0.001
4,2023-07-12 01:25:00+00:00,1340.6,5.0,1.3406,0.001
...,...,...,...,...,...
80,2023-07-12 07:45:00+00:00,478.0,0.0,0.0000,0.000
81,2023-07-12 07:50:00+00:00,478.0,0.0,0.0000,0.000
82,2023-07-12 07:55:00+00:00,478.0,0.0,0.0000,0.000
83,2023-07-12 08:00:00+00:00,468.4,0.0,0.0000,0.000


In [14]:
recalculating_optimizer = RecalculatingWattTimeOptimizer(
    region=region, 
    watttime_username=username, 
    watttime_password=password, 
    usage_time_required_minutes=240,
    usage_power_kw=2,
    optimization_method="simple"
)

recalculating_optimizer.get_new_schedule(
        window_start, 
        window_end
    )

recalculating_optimizer.get_combined_schedule().reset_index()

240
2
== Simple fit! ==


,point_time,pred_moer,usage,emissions_co2e_lb,energy_usage_mwh
0,2023-07-11 20:10:00+00:00,1357.4,0.0,0.000000,0.000000
1,2023-07-11 20:15:00+00:00,1357.4,0.0,0.000000,0.000000
2,2023-07-11 20:20:00+00:00,1357.6,0.0,0.000000,0.000000
3,2023-07-11 20:25:00+00:00,1357.4,0.0,0.000000,0.000000
4,2023-07-11 20:30:00+00:00,1355.9,0.0,0.000000,0.000000
...,...,...,...,...,...
139,2023-07-12 07:45:00+00:00,478.0,5.0,0.079667,0.000167
140,2023-07-12 07:50:00+00:00,478.0,5.0,0.079667,0.000167
141,2023-07-12 07:55:00+00:00,478.0,5.0,0.079667,0.000167
142,2023-07-12 08:00:00+00:00,468.4,5.0,0.078067,0.000167
